In [13]:
import torch
import pandas as pd
from src.main import main
from transformers import pipeline
from sentence_transformers import SentenceTransformer

In [ ]:
datasets = ["li2022_EN_SS_trimmed_mean"]

In [ ]:
config = {
    "datasets": datasets,
    "model": "bert-base-uncased",
    "decoder": "brain_decoder",
    "loss": "mixco",
    "valid_ratio": 0.1,
    "test_ratio": 0.1,
    "context_length": 6,
    "lag": 3,
    "smooth": 6,
    "stack": 0,
    "dropout": 0.7,
    "patience": 20,
    "lr": 1e-4,
    "weight_decay": 1e-6,
    "batch_size": 1,
    "temperature": 0.05
}

In [ ]:
gpt2 = pipeline("text-generation", model="gpt2")

# Max number of tokens in a chunk

In [ ]:
clone_config = config.copy()
clone_config["context_length"] = 0
df_train, df_valid, df_test = main(return_data=True, caching=False, **clone_config)

In [27]:
num_tokens = []
for df in [df_train, df_valid, df_test]:
    for _, row in df.drop_duplicates(["dataset", "run"]).iterrows():
        for chunk in row.text:
            num_tokens.append([row.dataset, row.run, len(gpt2.tokenizer.encode(chunk))])
num_tokens = pd.DataFrame(num_tokens, columns=["dataset", "run", "num_tokens"])
num_tokens = num_tokens.sort_values("run")

In [24]:
import plotly.express as px
px.histogram(num_tokens, x="num_tokens", facet_col="run", marginal="box", nbins=50)

# Fetch data and decoder

In [ ]:
df_train, df_valid, df_test = main(return_data=True, caching=False, **config)
_, decoder = main(**config)

# Decode

In [ ]:
_, row = next(iter(df_train.iterrows()))
with torch.no_grad():
    predicted_latent = decoder(decoder.projector[row.dataset + "/" + row.subject](row.X))

In [ ]:
prompt = "Once upon a time"
num_sentences = 5

In [ ]:
gpt2(
    prompt,
    max_new_tokens=12,
    num_return_sequences=num_sentences,
    pad_token_id=50256,
)